In [1]:
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Add the scripts folder to the system path
import sys
sys.path.append("../scripts")

In [3]:
from data import load_data
from preprocessing import preprocess_data


In [5]:
# Data Manipulation and Analysis
import pandas as pd
import numpy as np

In [6]:
# Machine Learning Models and Tools
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [7]:
# Model Selection and Evaluation
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer

In [8]:
data = load_data('housing.csv')

Successfully loaded data from housing.csv.


drop NA values so that all rows have values

In [9]:
data.dropna(inplace=True)

In [10]:
#dividing into income categories

data["income_cat"] = np.ceil(data["median_income"] / 1.5)

#putting everything above the 5th category as the 5th category

data["income_cat"].where(data["median_income"] < 5, other=5.0 , inplace = True)

In [11]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=29)

for train_index, test_index in split.split(data, data["income_cat"]):
    strat_train_split = data.iloc[train_index]
    strat_test_split = data.iloc[test_index]


In [14]:
strat_train_set = data.iloc[train_index].copy()
strat_test_set = data.iloc[test_index].copy()

In [15]:
# Drop the temporary stratification column from both sets
for dataset in (strat_train_set, strat_test_set):
    dataset.drop("income_cat", axis=1, inplace=True)


Stratified sampling ensures that the distribution of key categories in the training and testing sets matches that of the original dataset. This is especially important for a medium-sized dataset like this, where certain categories may be sparse.

We remove the temporary income_cat column once the split is done, as it was only used for stratification.

Afterwards we prepare training and testing data (x_train, y_train, x_test, y_test) from the stratified splits

In [16]:
# Now define x and y from the clean training and testing sets
x_train = strat_train_set.drop(['median_house_value'], axis=1)
y_train = strat_train_set['median_house_value']
x_test = strat_test_set.drop(['median_house_value'], axis=1)
y_test = strat_test_set['median_house_value']

joinng x and y training data so we can analyse some basic corrolations 

In [17]:
train_data = x_train.join(y_train)

In [18]:
train_data['total_rooms'] = np.log(train_data['total_rooms'] + 1) 
train_data['total_bedrooms'] = np.log(train_data['total_bedrooms'] + 1)
train_data['population'] = np.log(train_data['population'] + 1)
train_data['households'] = np.log(train_data['households'] + 1)

In [19]:
x_test['total_rooms'] = np.log(x_test['total_rooms'] + 1) 
x_test['total_bedrooms'] = np.log(x_test['total_bedrooms'] + 1)
x_test['population'] = np.log(x_test['population'] + 1)
x_test['households'] = np.log(x_test['households'] + 1)

In [20]:
# Perform one-hot encoding on the 'ocean_proximity' column with `drop_first` to avoid multicollinearity issues
train_data = pd.get_dummies(train_data, columns=['ocean_proximity'], drop_first=True)
x_test = pd.get_dummies(x_test, columns=['ocean_proximity'], drop_first=True)

In [21]:
coastline_points = [
    # Southern California
    (32.5343, -117.1232),  # Imperial Beach near San Diego (close to the border)
    (32.8525, -117.2728),  # La Jolla Shores, San Diego
    (33.5410, -117.7854),  # Laguna Beach
    (34.0092, -118.4976),  # Santa Monica Pier
    (34.0259, -118.7798),  # Malibu Lagoon State Beach

    # Central Coast
    (34.4140, -119.6846),  # East Beach, Santa Barbara
    (35.2819, -120.6596),  # Avila Beach, San Luis Obispo
    (36.5552, -121.9233),  # Point Lobos, Carmel Highlands
    (36.9514, -122.0263),  # Santa Cruz Beach

    # Bay Area
    (37.7735, -122.5154),  # Ocean Beach, San Francisco
    (37.8299, -122.4194),  # Baker Beach, San Francisco
    (38.0573, -122.7562),  # Dillon Beach, Marin County

    # Northern California
    (39.3086, -123.8102),  # Fort Bragg, Mendocino County
    (40.4358, -124.3945),  # Cape Mendocino
    (41.7450, -124.1835)   # Crescent City Beach (near Oregon border)
]

In [22]:
# Preprocess data
x_train_fe, y_train, preprocessing_pipeline = preprocess_data(train_data, coastline_points)

In [23]:
# Align columns in x_train and x_test by adding missing columns with NaN values
missing_in_train = set(x_test.columns) - set(x_train_fe.columns)
missing_in_test = set(x_train_fe.columns) - set(x_test.columns)

# Add missing columns to x_train and x_test with NaN values
for col in missing_in_train:
    x_train_fe[col] = np.nan

for col in missing_in_test:
    x_test[col] = np.nan

# Reorder columns so that they match
x_train = x_train_fe[x_test.columns]


In [24]:
# Join x_train with y_train, compute correlations, and sort by the target correlation in descending order
correlation_check = x_train.join(y_train).corr()['median_house_value'].abs().sort_values(ascending=False)

# Display the sorted correlations
print(correlation_check)

median_house_value                1.000000
residential_density_per_capita    0.689287
median_income                     0.687044
income_to_room_ratio              0.674743
income_per_household              0.650456
median_age_interaction            0.590842
high_income_area                  0.514883
dist_to_coast                     0.499609
ocean_proximity_INLAND            0.486948
population_density_per_room       0.282827
rooms_per_person                  0.278270
income_per_age_of_housing         0.205263
ocean_proximity_NEAR BAY          0.168867
population_per_household          0.160798
bedroom_ratio                     0.153434
total_rooms                       0.151902
latitude                          0.143206
ocean_proximity_NEAR OCEAN        0.133778
household_room_interaction        0.121947
housing_median_age                0.108786
population_per_bedroom            0.107503
lat_long_interaction              0.103363
rooms_per_household               0.081739
bedrooms_pe

In [25]:
# **Full Pipeline**: Including both preprocessing and model
full_pipeline = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('model', LinearRegression())
])

# Fit the model on the preprocessed data
full_pipeline.fit(x_train, y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('outlier_handler',
                                                                   OutlierHandler()),
                                                                  ('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population', 'households',
                                              

In [26]:
y_pred = full_pipeline.predict(x_test) 

Training 

We also need to splitt data, we do not ened to do the train/test split again but me must do the x/y split again
Also we must  add the new features to the test_data to evaluate the model 

not doing hyperparameter training for linear regression, saying it is okay for it to go into production

this is the same except we scale our data
we only need to scale the input, not the output

a different model to linear regression, Random Forest Regressor
ensemble is a different type of mL where we combine multiple types of models
    in this case decision trees

In [31]:
import importlib
import model_training
importlib.reload(model_training)
from model_training import get_model_pipelines, evaluate_model


In [32]:
pipelines = get_model_pipelines(preprocessing_pipeline)


In [34]:
# Train and evaluate models
results = {}

In [35]:
for model_name, pipeline in pipelines.items():
    print(f"\nTraining and evaluating {model_name}...")

 
    pipeline.fit(x_train, y_train)  # Train the model
    y_pred = pipeline.predict(x_test) 

    print(f"Evaluation for {model_name}:")
    metrics = evaluate_model(y_test, y_pred)  # Evaluate predictions
    
    # Store results for comparison
    results[model_name] = metrics



Training and evaluating Linear Regression...
Evaluation for Linear Regression:
Mean Squared Error: 11567652137708.57
Root Mean Squared Error: 3401125.1282051606
Mean Absolute Error: 2880969.0080079776
R² Score: -855.9111855796936

Training and evaluating Random Forest...
Evaluation for Random Forest:
Mean Squared Error: 12872584165.182417
Root Mean Squared Error: 113457.41123955903
Mean Absolute Error: 88170.68535845361
R² Score: 0.046421760687047064

Training and evaluating Gradient Boosting...
Evaluation for Gradient Boosting:
Mean Squared Error: 12609823576.707005
Root Mean Squared Error: 112293.47076614475
Mean Absolute Error: 83052.05500948447
R² Score: 0.06588659976706146

Training and evaluating XGBoost...
Evaluation for XGBoost:
Mean Squared Error: 12183502403.439632
Root Mean Squared Error: 110378.9037970555
Mean Absolute Error: 83520.2620217305
R² Score: 0.0974677173243047

Training and evaluating K-Nearest Neighbors...
Evaluation for K-Nearest Neighbors:
Mean Squared Error:

/Users/favourchianumba/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Evaluation for Huber Regressor:
Mean Squared Error: 7607470312845.036
Root Mean Squared Error: 2758164.301278123
Mean Absolute Error: 2729654.4784738715
R² Score: -562.5479289519587

Training and evaluating LassoCV...


/Users/favourchianumba/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26297856198.742188, tolerance: 17560817085.315014
  model = cd_fast.enet_coordinate_descent_gram(
/Users/favourchianumba/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 26285945923.484375, tolerance: 17560817085.315014
  model = cd_fast.enet_coordinate_descent_gram(
/Users/favourchianumba/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25818148715.09375, tolerance: 17560817085.315014
  model = cd_fast.en

Evaluation for LassoCV:
Mean Squared Error: 5235640440080.958
Root Mean Squared Error: 2288152.1890121205
Mean Absolute Error: 2244016.024619193
R² Score: -386.84697217455283

Training and evaluating RidgeCV...
Evaluation for RidgeCV:
Mean Squared Error: 10340571664797.008
Root Mean Squared Error: 3215675.9265816896
Mean Absolute Error: 2817742.9483720483
R² Score: -765.0112371436032

Training and evaluating MLP Regressor...


/Users/favourchianumba/opt/miniconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Evaluation for MLP Regressor:
Mean Squared Error: 4443167525810.266
Root Mean Squared Error: 2107882.2371779373
Mean Absolute Error: 2077751.2996709428
R² Score: -328.14198205008216

Training and evaluating SGDRegressor...
Evaluation for SGDRegressor:
Mean Squared Error: 3993376891668.5063
Root Mean Squared Error: 1998343.5369496674
Mean Absolute Error: 1981435.2186629982
R² Score: -294.822288392575


In [36]:
# Compare models
comparison_df = pd.DataFrame(results).T
print("\nModel Comparison:")
print(comparison_df)


Model Comparison:
                                    MSE          RMSE           MAE  \
Linear Regression          1.156765e+13  3.401125e+06  2.880969e+06   
Random Forest              1.287258e+10  1.134574e+05  8.817069e+04   
Gradient Boosting          1.260982e+10  1.122935e+05  8.305206e+04   
XGBoost                    1.218350e+10  1.103789e+05  8.352026e+04   
K-Nearest Neighbors        1.276676e+10  1.129901e+05  9.115505e+04   
Support Vector Regression  1.431147e+10  1.196306e+05  8.953514e+04   
ElasticNet                 2.523375e+10  1.588514e+05  1.242216e+05   
LightGBM                   1.157495e+10  1.075870e+05  8.015793e+04   
Huber Regressor            7.607470e+12  2.758164e+06  2.729654e+06   
LassoCV                    5.235640e+12  2.288152e+06  2.244016e+06   
RidgeCV                    1.034057e+13  3.215676e+06  2.817743e+06   
MLP Regressor              4.443168e+12  2.107882e+06  2.077751e+06   
SGDRegressor               3.993377e+12  1.998344e+06  1.9

In [ ]:
print("First few actual values (y_test):", y_test[:5])
print("First few predicted values (y_pred):", y_pred[:5])


Based on the updated evaluation metrics, the LightGBM model demonstrates the best performance among the tested models:

***LightGBM:***

Mean Squared Error (MSE): 11,574,950,000.00

Root Mean Squared Error (RMSE): 107,587.00

Mean Absolute Error (MAE): 80,157.93

R² Score: 0.143

This model has the lowest MSE, RMSE, and MAE among the models evaluated, as well as the highest R^2 score (0.143). R^2 value indicates that the model explains a modest proportion of the variance in the target, it outperforms alternatives like Gradient Boosting and XGBoost. These results suggest that LightGBM captures the target variable's underlying patterns more effectively than other models, though further improvements may be needed to enhance predictive performance across all models.
 

In [ ]:
# Create the LightGBM Regressor pipeline
pipeline_lgb = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),  # your existing preprocessing pipeline
    ('model', lgb.LGBMRegressor(random_state=42))  # model to tune
])

# Define the refined parameter grid for LightGBM
param_dist_lgb2 = {
    'model__n_estimators': [700, 800, 900, 1000],  # increased number of boosting stages
    'model__learning_rate': [0.06, 0.08, 0.1],  # fine-tuned learning rates
    'model__num_leaves': [60, 70, 80],  # close to best range found before
    'model__max_depth': [12, 14, 16],  # fine-tuning around the best depth
    'model__min_data_in_leaf': [10, 15, 20],  # keeping min_data_in_leaf for balance
    'model__subsample': [0.85, 0.9, 0.95, 1.0],  # adjust subsampling
    'model__colsample_bytree': [0.5, 0.55, 0.6, 0.7],  # slightly lower feature sampling
    'model__reg_alpha': [0.05, 0.1, 0.2],  # regularization parameter for potential minor improvements
    'model__reg_lambda': [0.05, 0.1, 0.2]  # regularization parameter fine-tuning
}

# Set up RandomizedSearchCV for LightGBM with adjusted search space
random_search_lgb2 = RandomizedSearchCV(
    estimator=pipeline_lgb,  # the pipeline
    param_distributions=param_dist_lgb2,  # refined hyperparameter grid
    n_iter=30,  # reducing iterations to manage time, focusing on refined params
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Use all available CPU cores
    random_state=42,
    scoring='neg_mean_squared_error'  # minimize mean squared error
)

# Fit the RandomizedSearchCV
random_search_lgb2.fit(x_train, y_train)

# Print the best parameters and best score from RandomizedSearchCV
print("Best Parameters for Refined LightGBM:", random_search_lgb2.best_params_)
print("Best Score (Negative MSE):", random_search_lgb2.best_score_)

# Evaluate the model using the best parameters
best_lgb_model2 = random_search_lgb2.best_estimator_

# Predictions with the best model
lgb_predictions2 = best_lgb_model2.predict(x_test)

# Evaluate the model's performance
evaluate_model(y_test, lgb_predictions2)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10

[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing wa

{'MSE': 8706123218.926088,
 'RMSE': 93306.60865622589,
 'MAE': 70569.80402369787,
 'R2': 0.3550658093345229}

***Hyperparameter Tuning for LightGBM**

we can try increase the accuracy of the mondel by giving it a parameter grid and trying the different options 

lets use grid search cross validation

splitting it into k-folds, use all but 1 fold to train thedata and 1 for the validation 
and then evaluate the parameters we used

when we provide a parameter grid it is going to do the cross validation with all the different combinations

In [53]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV

# LightGBM pipeline with further narrowed parameter grid
pipeline_lgb_final = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),  # Existing preprocessing pipeline
    ('model', lgb.LGBMRegressor(random_state=42))  # Model to tune
])

# Focused parameter grid for LightGBM
param_dist_lgb_final = {
    'model__n_estimators': [600, 700, 800],  # Slightly increase n_estimators
    'model__learning_rate': [0.08, 0.1, 0.12],  # Fine-tune learning rate
    'model__num_leaves': [55, 60, 65],  # Adjust num_leaves around best value
    'model__max_depth': [10, 12, 14],  # Small tweaks to max depth
    'model__min_data_in_leaf': [10, 15, 20],  # Narrow down min_data_in_leaf
    'model__subsample': [0.85, 0.9, 0.95],  # Slight increase for subsampling
    'model__colsample_bytree': [0.55, 0.6, 0.65],  # Fine-tune colsample_bytree
    'model__reg_alpha': [0.05, 0.1, 0.15],  # Further narrow L1 regularization
    'model__reg_lambda': [0.05, 0.1, 0.15],  # Further narrow L2 regularization
}

# Set up RandomizedSearchCV for LightGBM
random_search_lgb_final = RandomizedSearchCV(
    estimator=pipeline_lgb_final,
    param_distributions=param_dist_lgb_final,
    n_iter=20,  # Smaller iteration count for final targeted tuning
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

# Fit the RandomizedSearchCV
random_search_lgb_final.fit(x_train, y_train)

# Print the best parameters and best score from RandomizedSearchCV
print("Best Parameters for Refined LightGBM:", random_search_lgb_final.best_params_)
print("Best Score (Negative MSE):", random_search_lgb_final.best_score_)

# Evaluate the model with the refined best parameters
best_lgb_model_final = random_search_lgb_final.best_estimator_

# Predictions with the final model
lgb_predictions_final = best_lgb_model_final.predict(x_test)

# Evaluate the model's performance
evaluate_model(y_test, lgb_predictions_final)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044789 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `fo

{'MSE': 8520732750.460398,
 'RMSE': 92307.81521875814,
 'MAE': 68179.37192644786,
 'R2': 0.3687992069364653}

In [48]:
# Refined Parameter Grid for Minor Adjustments
param_dist_lgb_refined = {
    'model__n_estimators': [600, 650, 700],
    'model__learning_rate': [0.07, 0.08],
    'model__num_leaves': [60, 65],
    'model__max_depth': [13, 14],
    'model__min_data_in_leaf': [10, 15],
    'model__subsample': [0.9, 1.0],
    'model__colsample_bytree': [0.5, 0.55],
    'model__reg_alpha': [0.05, 0.1],
    'model__reg_lambda': [0.1, 0.15]
}

# Setup RandomizedSearchCV for LightGBM with Refined Parameters
random_search_lgb_refined = RandomizedSearchCV(
    estimator=pipeline_lgb,  
    param_distributions=param_dist_lgb_refined, 
    n_iter=15,  # Limiting to 15 iterations for efficiency
    cv=5,
    verbose=1,
    n_jobs=-1,  
    random_state=42,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

# Fit the RandomizedSearchCV with refined parameters
random_search_lgb_refined.fit(x_train, y_train)

# Output best parameters and model evaluation
print("Best Parameters for Refined LightGBM:", random_search_lgb_refined.best_params_)
print("Best Score (Negative MSE):", random_search_lgb_refined.best_score_)

# Evaluate with best model
best_lgb_model_refined = random_search_lgb_refined.best_estimator_
lgb_predictions_refined = best_lgb_model_refined.predict(x_test)
evaluate_model(y_test, lgb_predictions_refined)


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15

[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored

{'MSE': 8221675497.879865,
 'RMSE': 90673.4553101395,
 'MAE': 70675.67584864603,
 'R2': 0.3909528386167963}

In [56]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb

# Create the LightGBM Regressor pipeline
pipeline_lgb = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),
    ('model', lgb.LGBMRegressor(random_state=42))
])

# Refined hyperparameter grid
param_dist_lgb_optimized = {
    'model__n_estimators': [600, 650, 700, 750],
    'model__learning_rate': [0.06, 0.07, 0.08],
    'model__num_leaves': [55, 60, 65],
    'model__max_depth': [12, 13, 14],
    'model__min_data_in_leaf': [8, 10, 15],
    'model__subsample': [0.85, 0.9, 1.0],
    'model__colsample_bytree': [0.5, 0.55, 0.6],
    'model__reg_alpha': [0.03, 0.05, 0.1],
    'model__reg_lambda': [0.08, 0.1, 0.15]
}

# Early stopping with a validation set
random_search_lgb_optimized = RandomizedSearchCV(
    estimator=pipeline_lgb,
    param_distributions=param_dist_lgb_optimized,
    n_iter=12,  # Fewer iterations for speed
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1,
    random_state=42,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

# Fit the model with early stopping
random_search_lgb_optimized.fit(x_train, y_train)

# Evaluate and display results
print("Best Parameters for Optimized LightGBM:", random_search_lgb_optimized.best_params_)
print("Best Score (Negative MSE):", random_search_lgb_optimized.best_score_)

# Use the best model for predictions and evaluation
best_lgb_model_optimized = random_search_lgb_optimized.best_estimator_
lgb_predictions_optimized = best_lgb_model_optimized.predict(x_test)
evaluate_model(y_test, lgb_predictions_optimized)


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10

[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. C

{'MSE': 8260403830.297441,
 'RMSE': 90886.76377942742,
 'MAE': 69985.77860997178,
 'R2': 0.38808391233405604}

In [74]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb

# Create the LightGBM Regressor pipeline
pipeline_lgb = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),
    ('model', lgb.LGBMRegressor(random_state=42))
])

# Further optimized hyperparameter grid
param_dist_lgb_optimized = {
    'model__n_estimators': [600, 650, 700],  # Narrower range
    'model__learning_rate': [0.065, 0.07, 0.075],  # Fine-tuned range
    'model__num_leaves': [60, 62, 65],  # Fewer options near the best
    'model__max_depth': [13, 14],  # Reduced range to balance complexity
    'model__min_data_in_leaf': [8, 10],  # Simplified for faster training
    'model__subsample': [0.85, 0.9],  # Optimized based on diminishing returns
    'model__colsample_bytree': [0.5, 0.55],  # Further restricted
    'model__reg_alpha': [0.04, 0.05],  # Fine-grained regularization
    'model__reg_lambda': [0.08, 0.1]  # Focused range
}

# RandomizedSearchCV with fewer iterations and folds for efficiency
random_search_lgb_optimized = RandomizedSearchCV(
    estimator=pipeline_lgb,
    param_distributions=param_dist_lgb_optimized,
    n_iter=8,  # Reduced for speed
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Utilize all available cores
    random_state=42,
    scoring='neg_mean_squared_error',  # Scoring metric
    return_train_score=True
)

# Fit the model with RandomizedSearchCV
random_search_lgb_optimized.fit(x_train, y_train)

# Display the best parameters and score
print("Best Parameters for Optimized LightGBM:", random_search_lgb_optimized.best_params_)
print("Best Score (Negative MSE):", random_search_lgb_optimized.best_score_)

# Use the best model for predictions and evaluation
best_lgb_model_optimized = random_search_lgb_optimized.best_estimator_
lgb_predictions_optimized = best_lgb_model_optimized.predict(x_test)

# Evaluate the model
results = evaluate_model(y_test, lgb_predictions_optimized)
print("Optimized Model Results:", results)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023011 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5932
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 w

In [75]:
param_dist_lgb_improved = {
    'model__n_estimators': [650, 700, 750],  # Slightly increased to explore a broader range
    'model__learning_rate': [0.06, 0.07, 0.075],  # Fine-tuned around previous best values
    'model__num_leaves': [58, 60, 62],  # Narrowed for better exploration around the optimal range
    'model__max_depth': [12, 13, 14],  # Maintaining similar depth but slightly narrowed
    'model__min_data_in_leaf': [8, 10, 12],  # Focus on smaller leaf sizes for more granularity
    'model__subsample': [0.9, 0.95, 1.0],  # Slightly more exploration at high subsampling levels
    'model__colsample_bytree': [0.5, 0.55, 0.6],  # Consistent with earlier ranges
    'model__reg_alpha': [0.03, 0.05, 0.08],  # Further tuned to minimize over-regularization
    'model__reg_lambda': [0.08, 0.1, 0.12],  # Adjusted for optimal balance between L2 penalties
}

random_search_lgb_improved = RandomizedSearchCV(
    estimator=pipeline_lgb,  
    param_distributions=param_dist_lgb_improved, 
    n_iter=15,  # Limited iterations for efficiency
    cv=5,  # 5-fold cross-validation for robustness
    verbose=1,
    n_jobs=-1,  
    random_state=42,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

# Fit the model
random_search_lgb_improved.fit(x_train, y_train)

# Output the best parameters and performance metrics
print("Best Parameters for Improved LightGBM:", random_search_lgb_improved.best_params_)
print("Best Score (Negative MSE):", random_search_lgb_improved.best_score_)

# Evaluate the best model
best_lgb_model_improved = random_search_lgb_improved.best_estimator_
lgb_predictions_improved = best_lgb_model_improved.predict(x_test)
results = evaluate_model(y_test, lgb_predictions_improved)
print("Improved Model Results:", results)


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was

In [63]:
param_dist_lgb_final = {
    'model__n_estimators': [600, 650],  # Focused on previously optimal range
    'model__learning_rate': [0.07, 0.08],  # Narrow for stability
    'model__num_leaves': [55, 60],  # Reduce upper limit
    'model__max_depth': [13, 14],  # Stick to optimal depths
    'model__min_data_in_leaf': [12, 15],  # More conservative lower bound
    'model__subsample': [0.9, 0.95],  # Slightly less aggressive
    'model__colsample_bytree': [0.5, 0.55],  # Balanced feature usage
    'model__reg_alpha': [0.05, 0.1],  # Return to stable range
    'model__reg_lambda': [0.1, 0.15]  # Prior optimal range
}

# Setup RandomizedSearchCV for LightGBM with the refined parameter grid
random_search_lgb_final = RandomizedSearchCV(
    estimator=pipeline_lgb,
    param_distributions=param_dist_lgb_final,
    n_iter=10,  # Reduced iterations for efficiency
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42,
    scoring='neg_mean_squared_error',
    return_train_score=True
)

# Fit the RandomizedSearchCV with refined parameters
random_search_lgb_final.fit(x_train, y_train)

# Evaluate with the best model
best_lgb_model_final = random_search_lgb_final.best_estimator_
lgb_predictions_final = best_lgb_model_final.predict(x_test)
evaluate_model(y_test, lgb_predictions_final)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored

{'MSE': 7968798345.296319,
 'RMSE': 89268.12614419729,
 'MAE': 68822.48559678528,
 'R2': 0.409685530268199}

In [93]:
# Define the pipeline with preprocessing and the model (without early stopping)
pipeline_lgb = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),
    ('model', lgb.LGBMRegressor(random_state=42, n_estimators=1000))
])

# Define the parameter grid for hyperparameter tuning
param_dist_lgb_refined = {
    'model__n_estimators': [650, 700, 750, 800],  # Number of boosting rounds
    'model__learning_rate': [0.055, 0.06, 0.065, 0.07],  # Learning rate
    'model__num_leaves': [56, 58, 60, 62],  # Number of leaves in a tree
    'model__max_depth': [12, 13, 14],  # Max depth of trees
    'model__min_data_in_leaf': [8, 10, 12, 15],  # Minimum samples per leaf
    'model__subsample': [0.85, 0.9, 0.95],  # Fraction of samples for training
    'model__colsample_bytree': [0.5, 0.55, 0.6],  # Fraction of features per tree
    'model__reg_alpha': [0.03, 0.04, 0.05, 0.08],  # L1 regularization
    'model__reg_lambda': [0.08, 0.1, 0.12, 0.15],  # L2 regularization
}

# Perform RandomizedSearchCV for hyperparameter tuning
random_search_lgb = RandomizedSearchCV(
    estimator=pipeline_lgb,
    param_distributions=param_dist_lgb_refined,
    n_iter=20,  # Number of parameter settings to try
    cv=5,  # Number of cross-validation folds
    verbose=1,  # Print progress
    n_jobs=-1,  # Use all available cores
    random_state=42,
    scoring='neg_mean_squared_error',  # Use negative MSE as the metric for evaluation
    return_train_score=True
)

# Fit the RandomizedSearchCV with the hyperparameter grid
random_search_lgb.fit(x_train, y_train)

# Get the best model and make predictions
best_lgb_model = random_search_lgb.best_estimator_
lgb_predictions = best_lgb_model.predict(x_test)

evaluate_model(y_test, lgb_predictions)



Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8

[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.046976 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise

{'MSE': 8694902549.548473,
 'RMSE': 93246.46132453754,
 'MAE': 71401.00300634037,
 'R2': 0.3558970166516955}

In [87]:
import lightgbm as lgb
print(lgb.__version__)


4.5.0


***Hyperparameter Tuning for XGBoost***

In [49]:
# Import necessary packages

from sklearn.pipeline import Pipeline

# Create the XGBoost Regressor pipeline
pipeline_xgb1 = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),  # your existing preprocessing pipeline
    ('model', XGBRegressor(random_state=42))  # model to tune
])

# Define the parameter grid for XGBoost
param_dist_xgb1 = {
    'model__n_estimators': [100, 200, 500],  # number of boosting rounds
    'model__learning_rate': [0.01, 0.05, 0.1, 0.2],  # shrinkage rate
    'model__max_depth': [4, 6, 8, 10],  # depth of trees
    'model__reg_lambda': [0.1, 1, 5],  # L2 regularization
    'model__reg_alpha': [0, 0.1, 1],  # L1 regularization
    'model__subsample': [0.6, 0.8, 1.0],  # fraction of samples used for each tree
    'model__colsample_bytree': [0.6, 0.8, 1.0],  # fraction of features used per tree
    'model__gamma': [0, 0.1, 0.3],  # minimum loss reduction required to make a split
    'model__min_child_weight': [1, 3, 5],  # minimum sum of instance weight in a child
}

# Set up RandomizedSearchCV for XGBoost
random_search_xgb1 = RandomizedSearchCV(
    estimator=pipeline_xgb1,  # the pipeline
    param_distributions=param_dist_xgb1,  # hyperparameter grid
    n_iter=20,  # number of parameter settings to sample
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Use all available CPU cores
    random_state=42,
    scoring='neg_mean_squared_error'  # we want to minimize mean squared error
)

# Fit the RandomizedSearchCV
random_search_xgb1.fit(x_train, y_train)

# Print the best parameters and best score from RandomizedSearchCV
print("Best Parameters for XGBoost:", random_search_xgb1.best_params_)
print("Best Score (Negative MSE):", random_search_xgb1.best_score_)

# Now evaluate the model using the best parameters
best_xgb_model1 = random_search_xgb1.best_estimator_

# Predictions with the best model
xgb_predictions1 = best_xgb_model1.predict(x_test)

# Evaluate the model's performance
evaluate_model(y_test, xgb_predictions1)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters for XGBoost: {'model__subsample': 1.0, 'model__reg_lambda': 0.1, 'model__reg_alpha': 0.1, 'model__n_estimators': 500, 'model__min_child_weight': 5, 'model__max_depth': 8, 'model__learning_rate': 0.05, 'model__gamma': 0, 'model__colsample_bytree': 0.6}
Best Score (Negative MSE): -2003294275.1888878
Mean Squared Error: 7786242333.929722
Root Mean Squared Error: 88239.68684174781
Mean Absolute Error: 69352.37428699229
R² Score: 0.42320895630768907


{'MSE': 7786242333.929722,
 'RMSE': 88239.68684174781,
 'MAE': 69352.37428699229,
 'R2': 0.42320895630768907}

In [50]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

# Create the XGBoost Regressor pipeline
pipeline_xgb_final = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),  # your existing preprocessing pipeline
    ('model', xgb.XGBRegressor(random_state=42, verbosity=0))  # model to tune
])

# Refined parameter grid for XGBoost
param_dist_xgb_final = {
    'model__n_estimators': [300, 400, 500, 600],  # Lower max to keep runtime reasonable
    'model__learning_rate': [0.03, 0.05, 0.07],  # Smaller steps for learning rate
    'model__max_depth': [6, 7, 8, 9],  # Narrower range to balance model complexity
    'model__min_child_weight': [3, 5, 7],  # Moderate values to avoid overfitting
    'model__subsample': [0.7, 0.8, 0.9, 1.0],  # Allow some flexibility in sampling
    'model__colsample_bytree': [0.5, 0.6, 0.7],  # Fraction of columns to use for training
    'model__gamma': [0, 0.1, 0.3, 0.5],  # Regularization term to reduce overfitting
    'model__reg_alpha': [0.05, 0.1, 0.2],  # L1 regularization
    'model__reg_lambda': [0.05, 0.1, 0.2],  # L2 regularization
}

# Set up RandomizedSearchCV for XGBoost with refined parameters
random_search_xgb_final = RandomizedSearchCV(
    estimator=pipeline_xgb_final,  # the pipeline
    param_distributions=param_dist_xgb_final,  # hyperparameter grid
    n_iter=20,  # Limit to 20 iterations for efficiency
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Use all available CPU cores
    random_state=42,
    scoring='neg_mean_squared_error'  # minimize MSE
)

# Fit the RandomizedSearchCV
random_search_xgb_final.fit(x_train, y_train)

# Print the best parameters and best score from RandomizedSearchCV
print("Best Parameters for XGBoost:", random_search_xgb_final.best_params_)
print("Best Score (Negative MSE):", random_search_xgb_final.best_score_)

# Now evaluate the model using the best parameters
best_xgb_model_final = random_search_xgb_final.best_estimator_

# Predictions with the best model
xgb_predictions_final = best_xgb_model_final.predict(x_test)

# Evaluate the model's performance
evaluate_model(y_test, xgb_predictions_final)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters for XGBoost: {'model__subsample': 0.9, 'model__reg_lambda': 0.05, 'model__reg_alpha': 0.05, 'model__n_estimators': 500, 'model__min_child_weight': 7, 'model__max_depth': 8, 'model__learning_rate': 0.03, 'model__gamma': 0, 'model__colsample_bytree': 0.6}
Best Score (Negative MSE): -1997366820.3996003
Mean Squared Error: 8375230323.522117
Root Mean Squared Error: 91516.28447179287
Mean Absolute Error: 71266.69162321079
R² Score: 0.37957776905850216


{'MSE': 8375230323.522117,
 'RMSE': 91516.28447179287,
 'MAE': 71266.69162321079,
 'R2': 0.37957776905850216}

In [76]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

# Create the XGBoost Regressor pipeline
pipeline_xgb = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),  # your existing preprocessing pipeline
    ('model', XGBRegressor(random_state=42))  # XGBoost model to tune
])

# Define a refined parameter grid for XGBoost
param_dist_xgb2 = {
    'model__n_estimators': [500, 550, 600],  # Slightly higher values to ensure sufficient boosting rounds
    'model__learning_rate': [0.02, 0.03, 0.04],  # Smaller increments around best
    'model__max_depth': [7, 8, 9],  # Depth slightly tuned around current best
    'model__min_child_weight': [6, 7, 8],  # Focusing near current optimum
    'model__subsample': [0.85, 0.9, 0.95],  # Subsampling around previous best values
    'model__colsample_bytree': [0.5, 0.55, 0.6],  # Slight adjustments in colsample_bytree
    'model__reg_alpha': [0.04, 0.05, 0.06],  # L1 regularization refined around best values
    'model__reg_lambda': [0.04, 0.05, 0.06],  # L2 regularization also refined
    'model__gamma': [0, 0.1, 0.2]  # Testing a few small values for gamma
}

# Set up RandomizedSearchCV for refined XGBoost tuning
random_search_xgb2 = RandomizedSearchCV(
    estimator=pipeline_xgb,  # the pipeline
    param_distributions=param_dist_xgb2,  # hyperparameter grid
    n_iter=15,  # Limiting the iterations for efficiency
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Use all available CPU cores
    random_state=42,
    scoring='neg_mean_squared_error'  # Minimizing mean squared error
)

# Fit the RandomizedSearchCV
random_search_xgb2.fit(x_train, y_train)

# Print the best parameters and best score from RandomizedSearchCV
print("Best Parameters for Refined XGBoost:", random_search_xgb2.best_params_)
print("Best Score (Negative MSE):", random_search_xgb2.best_score_)

# Now evaluate the model using the best parameters
best_xgb_model2 = random_search_xgb2.best_estimator_

# Predictions with the best model
xgb_predictions2 = best_xgb_model2.predict(x_test)

# Evaluate the model's performance
evaluate_model(y_test, xgb_predictions2)


Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best Parameters for Refined XGBoost: {'model__subsample': 0.95, 'model__reg_lambda': 0.04, 'model__reg_alpha': 0.05, 'model__n_estimators': 600, 'model__min_child_weight': 8, 'model__max_depth': 7, 'model__learning_rate': 0.04, 'model__gamma': 0, 'model__colsample_bytree': 0.5}
Best Score (Negative MSE): -1985241917.7242103
Mean Squared Error: 7636181829.56061
Root Mean Squared Error: 87385.24949647172
Mean Absolute Error: 65628.0461466999
R² Score: 0.4343251727340498


{'MSE': 7636181829.56061,
 'RMSE': 87385.24949647172,
 'MAE': 65628.0461466999,
 'R2': 0.4343251727340498}

In [77]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline

# Create the Random Forest Regressor pipeline
pipeline_rf = Pipeline(steps=[
    ('preprocessing', preprocessing_pipeline),  # your existing preprocessing pipeline
    ('model', RandomForestRegressor(random_state=42))  # model to tune
])

# Define the parameter grid for Random Forest
param_dist_rf = {
    'model__n_estimators': [100, 200, 300, 400, 500],  # number of trees
    'model__max_depth': [None, 10, 20, 30, 40],  # maximum depth of trees
    'model__min_samples_split': [2, 5, 10],  # min samples needed to split a node
    'model__min_samples_leaf': [1, 2, 4],  # min samples needed to be at a leaf node
    'model__max_features': ['auto', 'sqrt', 'log2'],  # max number of features used per tree
    'model__bootstrap': [True, False]  # whether bootstrap samples are used when building trees
}

# Set up RandomizedSearchCV for Random Forest
random_search_rf = RandomizedSearchCV(
    estimator=pipeline_rf,  # the pipeline
    param_distributions=param_dist_rf,  # hyperparameter grid
    n_iter=20,  # number of parameter settings to sample
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Use all available CPU cores
    random_state=42,
    scoring='neg_mean_squared_error'  # we want to minimize mean squared error
)

# Fit the RandomizedSearchCV
random_search_rf.fit(x_train, y_train)

# Print the best parameters and best score from RandomizedSearchCV
print("Best Parameters for Random Forest:", random_search_rf.best_params_)
print("Best Score (Negative MSE):", random_search_rf.best_score_)

# Now evaluate the model using the best parameters
best_rf_model = random_search_rf.best_estimator_

# Predictions with the best model
rf_predictions = best_rf_model.predict(x_test)

# Evaluate the model's performance
evaluate_model(y_test, rf_predictions)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters for Random Forest: {'model__n_estimators': 500, 'model__min_samples_split': 2, 'model__min_samples_leaf': 2, 'model__max_features': 'auto', 'model__max_depth': 20, 'model__bootstrap': True}
Best Score (Negative MSE): -2317868306.416954
Mean Squared Error: 13033682947.82039
Root Mean Squared Error: 114165.15645248505
Mean Absolute Error: 88508.55683943066
R² Score: 0.034487848153859746


{'MSE': 13033682947.82039,
 'RMSE': 114165.15645248505,
 'MAE': 88508.55683943066,
 'R2': 0.034487848153859746}

Saving the best weights and model

In [96]:
best_lgb_model_final = best_lgb_model_improved
best_xgb_model_final = best_xgb_model2

# Get predictions from the best models
lgb_test_preds = best_lgb_model_final.predict(x_test)
xgb_test_preds = best_xgb_model_final.predict(x_test)


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10


In [97]:
import itertools

# Define the range of weights to test
weight_range = np.linspace(0.1, 0.9, 9)  # 0.1 to 0.9 in steps of 0.1

# Store the results for each weight combination
best_weight = None
best_r2 = -np.inf  # Start with the lowest possible R²
best_metrics = None

print("Tuning weights for Weighted Averaging Ensemble...")
for weight in weight_range:
    # Define weights for LightGBM and XGBoost
    weights = [weight, 1 - weight]

    # Compute weighted predictions
    weighted_predictions = (
        weights[0] * lgb_test_preds + weights[1] * xgb_test_preds
    )

    # Evaluate performance
    mse = mean_squared_error(y_test, weighted_predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, weighted_predictions)
    r2 = r2_score(y_test, weighted_predictions)

    # Update the best weights if R² improves
    if r2 > best_r2:
        best_r2 = r2
        best_weight = weights
        best_metrics = (mse, rmse, mae, r2)

# Print the best weights and corresponding performance metrics
print("\nBest Weights for Weighted Averaging:")
print(f"LightGBM Weight: {best_weight[0]}")
print(f"XGBoost Weight: {best_weight[1]}")
print("\nBest Weighted Averaging Ensemble Performance:")
print(f"Mean Squared Error: {best_metrics[0]}")
print(f"Root Mean Squared Error: {best_metrics[1]}")
print(f"Mean Absolute Error: {best_metrics[2]}")
print(f"R² Score: {best_metrics[3]}")


Tuning weights for Weighted Averaging Ensemble...

Best Weights for Weighted Averaging:
LightGBM Weight: 0.1
XGBoost Weight: 0.9

Best Weighted Averaging Ensemble Performance:
Mean Squared Error: 7643157045.648476
Root Mean Squared Error: 87425.15110452184
Mean Absolute Error: 65597.59024617601
R² Score: 0.4338084610784456


MSE and RMSE are relatively high, indicating the model's predictions are still somewhat far from the actual values, but the error is not unreasonably large.

MAE suggests that the average prediction error in the units of the target variable is around 65,600, which is a useful indicator of the model's predictive accuracy.

R² Score of 0.434 means that about 43.4% of the variance in the target variable is explained by the weighted averaging ensemble, which is a good result, but there's still room for improvement

In [99]:
import pickle
import os

# Define the path to the models folder (relative to the current working directory)
models_dir = os.path.join(os.getcwd(), 'models')

# Ensure the models directory exists
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# Finalize the optimal weights
optimal_weights = [0.9, 0.1]

# Final weighted predictions
final_weighted_predictions = (
    optimal_weights[0] * lgb_test_preds + optimal_weights[1] * xgb_test_preds
)

# Evaluate and save the performance
print("Final Weighted Averaging Ensemble Performance:")
evaluate_model(y_test, final_weighted_predictions)

# Save the models and weights for reproducibility
with open(os.path.join(models_dir, 'best_lgb_model_final.pkl'), 'wb') as f:
    pickle.dump(best_lgb_model_final, f)

with open(os.path.join(models_dir, 'best_xgb_model_final.pkl'), 'wb') as f:
    pickle.dump(best_xgb_model_final, f)

with open(os.path.join(models_dir, 'optimal_weights.pkl'), 'wb') as f:
    pickle.dump(optimal_weights, f)

print("Finalized models and weights saved.")


Final Weighted Averaging Ensemble Performance:
Mean Squared Error: 7832493630.876227
Root Mean Squared Error: 88501.3764349246
Mean Absolute Error: 65959.85542175549
R² Score: 0.4197827421348207
Finalized models and weights saved.


In [101]:
import pickle
import os

# Define the path to the models folder
models_dir = os.path.join(os.getcwd(), 'models')

# Load the saved models and weights
with open(os.path.join(models_dir, 'best_lgb_model_final.pkl'), 'rb') as f:
    best_lgb_model = pickle.load(f)

with open(os.path.join(models_dir, 'best_xgb_model_final.pkl'), 'rb') as f:
    best_xgb_model = pickle.load(f)

with open(os.path.join(models_dir, 'optimal_weights.pkl'), 'rb') as f:
    optimal_weights = pickle.load(f)


1. Averaging Ensemble (Simple)
In an averaging ensemble, we simply average the predictions of both models and evaluate the performance. This is the simplest approach and can often work well.

In [107]:

# Fit the best LightGBM model on the entire training set
best_lgb_model_final.fit(x_train, y_train)

# Fit the best XGBoost model on the entire training set
best_xgb_model_final.fit(x_train, y_train)

# Make predictions with each model
ensemble_lgb_predictions = best_lgb_model_final.predict(x_test)
ensemble_xgb_predictions = best_xgb_model_final.predict(x_test)

# Create ensemble predictions by averaging the predictions
ensemble_predictions = (ensemble_lgb_predictions + ensemble_xgb_predictions) / 2

# Evaluate the ensemble
mse = mean_squared_error(y_test, ensemble_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, ensemble_predictions)
r2 = r2_score(y_test, ensemble_predictions)

# Print evaluation metrics
print("Ensemble Model Performance:")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004090 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5933
[LightGBM] [Info] Number of data points in the train set: 16346, number of used features: 31
[LightGBM] [Info] Start training from score 206504.738774
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
Ensemble Model Performance:
Mean Squared Error: 7708150471.362157
Root Mean Squared Error: 87796.07321151759
Mean Absolute Error: 65656.5312277141
R² Score: 0.4289938631963357


In [111]:
# Assign weights based on model performance (adjust based on results)
weight_lgb = 0.01  # Assign higher weight if LightGBM performs better
weight_xgb = 0.99

# Create weighted predictions
weighted_ensemble_predictions = (
    weight_lgb * ensemble_lgb_predictions +
    weight_xgb * ensemble_xgb_predictions
)

# Evaluate weighted ensemble
mse = mean_squared_error(y_test, weighted_ensemble_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, weighted_ensemble_predictions)
r2 = r2_score(y_test, weighted_ensemble_predictions)

# Print evaluation metrics
print("Weighted Ensemble Model Performance:")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")


Weighted Ensemble Model Performance:
Mean Squared Error: 7636712351.057664
Root Mean Squared Error: 87388.28497606337
Mean Absolute Error: 65624.24780258266
R² Score: 0.43428587264103136


2. Stacking Ensemble (Advanced)
In stacking, you train a "meta-model" on the predictions of the base models (i.e., XGBoost and LightGBM in this case). The base models' predictions become input features for the meta-model. Typically, you use a simple linear model (like Ridge or Logistic Regression) as the meta-model.

In [103]:
from sklearn.linear_model import Ridge

# Get predictions from both base models (LightGBM and XGBoost)
lgb_train_preds = best_lgb_model_final.predict(x_train)
xgb_train_preds = best_xgb_model_final.predict(x_train)

# Stack predictions together to form a new dataset
stacked_train_preds = np.vstack((lgb_train_preds, xgb_train_preds)).T

# Use a simple Ridge Regression as the meta-model
meta_model = Ridge()

# Train the meta-model on the predictions of the base models
meta_model.fit(stacked_train_preds, y_train)

# Now make predictions using both models and the meta-model
lgb_test_preds = best_lgb_model_final.predict(x_test)
xgb_test_preds = best_xgb_model_final.predict(x_test)

# Stack the predictions of both models for the test set
stacked_test_preds = np.vstack((lgb_test_preds, xgb_test_preds)).T

# Make the final predictions using the meta-model
final_predictions = meta_model.predict(stacked_test_preds)

# Evaluate the performance of the stacked model
mse = mean_squared_error(y_test, final_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, final_predictions)
r2 = r2_score(y_test, final_predictions)

# Print evaluation metrics
print("Stacked Model Performance:")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
Stacked Model Performance:
Mean Squared Error: 8315574318.737226
Root Mean Squared Error: 91189.77091065218
Mean Absolute Error: 67420.35848016007
R² Score: 0.3839969802500738


In [105]:
from sklearn.model_selection import GridSearchCV


In [106]:
# **1. Stacking Ensemble with Ridge Meta-Model**
# Get predictions from both base models (LightGBM and XGBoost)
lgb_train_preds = best_lgb_model_final.predict(x_train)
xgb_train_preds = best_xgb_model_final.predict(x_train)

# Stack predictions together to form a new dataset
stacked_train_preds = np.vstack((lgb_train_preds, xgb_train_preds)).T

# Hyperparameter tuning for Ridge Regression as the meta-model
param_grid_meta = {
    'alpha': [0.01, 0.1, 1, 10, 100]  # Ridge regularization strength
}
grid_meta = GridSearchCV(
    Ridge(),
    param_grid_meta,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)
grid_meta.fit(stacked_train_preds, y_train)

# Best Ridge meta-model
best_meta_model = grid_meta.best_estimator_

# Train the tuned meta-model on the stacked predictions
best_meta_model.fit(stacked_train_preds, y_train)

# Now make predictions using both models and the tuned meta-model
lgb_test_preds = best_lgb_model_final.predict(x_test)
xgb_test_preds = best_xgb_model_final.predict(x_test)

# Stack the predictions of both models for the test set
stacked_test_preds = np.vstack((lgb_test_preds, xgb_test_preds)).T

# Make the final predictions using the meta-model
final_predictions = best_meta_model.predict(stacked_test_preds)

# Evaluate the performance of the stacked model
print("Tuned Stacking Model Performance:")
evaluate_model(y_test, final_predictions)

# **2. Weighted Averaging Ensemble**
# Adjust weights to prioritize LightGBM over XGBoost
weights = [0.6, 0.4]

# Compute weighted predictions
averaged_predictions_weighted = (
    weights[0] * lgb_test_preds + weights[1] * xgb_test_preds
)

# Evaluate the weighted averaging ensemble
print("Weighted Averaging Ensemble Performance:")
evaluate_model(y_test, averaged_predictions_weighted)

[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
Fitting 5 folds for each of 5 candidates, totalling 25 fits
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
Tuned Stacking Model Performance:
Mean Squared Error: 8315574318.620474
Root Mean Squared Error: 91189.77091001201
Mean Absolute Error: 67420.35847974404
R² Score: 0.38399698025872253
Weighted Averaging Ensemble Performance:
Mean Squared Error: 7733672184.015376
Root Mean Squared Error: 87941.2996493421
Mean Absolute Error: 65707.59330526237
R² Score: 0.42710325991856224


{'MSE': 7733672184.015376,
 'RMSE': 87941.2996493421,
 'MAE': 65707.59330526237,
 'R2': 0.42710325991856224}

In [121]:
import itertools


# Define the range of weights to test
weight_range = np.linspace(0.1, 0.9, 9)  # Test weights from 0.1 to 0.9

best_weight_combination = None
best_r2 = -np.inf
best_metrics = None

# Iterate through weight combinations for LightGBM and XGBoost
for weight in weight_range:
    # Define weights for LightGBM and XGBoost
    weights = [weight, 1 - weight]
    
    # Weighted train predictions
    weighted_train_preds = (
        weights[0] * lgb_train_preds + weights[1] * xgb_train_preds
    )
    
    # Fit the Ridge meta-model on weighted train predictions
    param_grid_meta = {'alpha': [0.01, 0.1, 1, 10, 100]}
    grid_meta = GridSearchCV(
        Ridge(),
        param_grid_meta,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )
    grid_meta.fit(weighted_train_preds.reshape(-1, 1), y_train)
    best_meta_model = grid_meta.best_estimator_
    best_meta_model.fit(weighted_train_preds.reshape(-1, 1), y_train)
    
    # Weighted test predictions
    weighted_test_preds = (
        weights[0] * lgb_test_preds + weights[1] * xgb_test_preds
    )
    
    # Stack test predictions for final predictions
    stacked_test_preds = weighted_test_preds.reshape(-1, 1)
    final_predictions = best_meta_model.predict(stacked_test_preds)
    
    # Evaluate performance
    metrics = evaluate_model(y_test, final_predictions)
    r2 = metrics['R2']
    
    # Track the best weight combination
    if r2 > best_r2:
        best_r2 = r2
        best_weight_combination = weights
        best_metrics = metrics

# Print the best weights and corresponding performance
print("\nBest Weights for Weighted Stacking Ensemble:")
print(f"LightGBM Weight: {best_weight_combination[0]}")
print(f"XGBoost Weight: {best_weight_combination[1]}")
print("\nBest Weighted Stacking Ensemble Performance:")
print(best_metrics)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits

Best Weights for Weighted Stacking Ensemble:
LightGBM Weight: 0.1
XGBoost Weight: 0.9

Best Weighted Stacking Ensemble Performance:
{'MSE': 7505194082.965343, 'RMSE': 86632.52324020895, 'MAE': 64812.11318648397, 'R2': 0.444028513039855}


In [122]:
# Define the range of weights to test for LightGBM and XGBoost
weight_range = np.linspace(0.1, 0.9, 9)  # Test weights from 0.1 to 0.9 for LightGBM

# Initialize variables to track the best results
best_weight_combination = None
best_r2 = -np.inf
best_metrics = None

# Loop through weight combinations for LightGBM and XGBoost
for weight in weight_range:
    # Define weights for LightGBM and XGBoost (LightGBM gets weight, XGBoost gets 1 - weight)
    weights = [weight, 1 - weight]
    
    # Create weighted train predictions by applying weights
    weighted_train_preds = (
        weights[0] * lgb_train_preds + weights[1] * xgb_train_preds
    )
    
    # Define a range of alpha values for Ridge regularization (try a wider range of values)
    param_grid_meta = {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100]  # Wider range for more flexibility
    }
    
    # Perform GridSearchCV to tune the Ridge model
    grid_meta = GridSearchCV(
        Ridge(),
        param_grid_meta,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )
    
    # Fit the Ridge meta-model on the weighted training predictions
    grid_meta.fit(weighted_train_preds.reshape(-1, 1), y_train)
    best_meta_model = grid_meta.best_estimator_
    best_meta_model.fit(weighted_train_preds.reshape(-1, 1), y_train)
    
    # Create weighted test predictions using the same weights
    weighted_test_preds = (
        weights[0] * lgb_test_preds + weights[1] * xgb_test_preds
    )
    
    # Stack the test predictions for final output
    stacked_test_preds = weighted_test_preds.reshape(-1, 1)
    final_predictions = best_meta_model.predict(stacked_test_preds)
    
    # Evaluate the model's performance
    metrics = evaluate_model(y_test, final_predictions)
    r2 = metrics['R2']
    
    # Track the best weight combination and performance based on R² score
    if r2 > best_r2:
        best_r2 = r2
        best_weight_combination = weights
        best_metrics = metrics

# Print the best weight combination and corresponding performance metrics
print("\nBest Weights for Weighted Stacking Ensemble:")
print(f"LightGBM Weight: {best_weight_combination[0]}")
print(f"XGBoost Weight: {best_weight_combination[1]}")
print("\nBest Weighted Stacking Ensemble Performance:")
print(best_metrics)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Fitting 5 folds for each of 6 candidates, totalling 30 fits

Best Weights for Weighted Stacking Ensemble:
LightGBM Weight: 0.1
XGBoost Weight: 0.9

Best Weighted Stacking Ensemble Performance:
{'MSE': 7505194082.965343, 'RMSE': 86632.52324020895, 'MAE': 64812.11318648397, 'R2': 0.444028513039855}


Performance Improvement: With an R² score of around 0.44, the model is capturing a fair amount of variance in the target variable. While it's not perfect, the ensemble method has clearly improved performance compared to individual models, and it is a good starting point for further optimization.

***1. Using Neural Networks as the Meta-Model:***
I replace the Ridge Regression meta-model with a Neural Network, using MLPRegressor from scikit-learn

***2. Boosting as the Meta-Model:***
Instead of a Ridge regression, I use a boosting method like XGBoost or LightGBM as the meta-model in the stacking ensemble. These models can capture more complex relationships in the data and may improve overall performance.

In [123]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the range of weights to test for LightGBM and XGBoost
weight_range = np.linspace(0.1, 0.9, 9)  # Test weights from 0.1 to 0.9 for LightGBM

# Initialize variables to track the best results
best_weight_combination = None
best_r2 = -np.inf
best_metrics = None

# Loop through weight combinations for LightGBM and XGBoost
for weight in weight_range:
    # Define weights for LightGBM and XGBoost (LightGBM gets weight, XGBoost gets 1 - weight)
    weights = [weight, 1 - weight]
    
    # Create weighted train predictions by applying weights
    weighted_train_preds = (
        weights[0] * lgb_train_preds + weights[1] * xgb_train_preds
    )
    
    # **Option 1: Use a Neural Network as the Meta-Model (MLPRegressor)**
    # Initialize the neural network regressor (MLP with 1 hidden layer, you can tune it further)
    nn_meta_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
    
    # Fit the neural network on the weighted training predictions
    nn_meta_model.fit(weighted_train_preds.reshape(-1, 1), y_train)
    
    # **Option 2: Use XGBoost as the Meta-Model**
    # Initialize an XGBoost model as the meta-learner
    xgb_meta_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
    
    # Fit the XGBoost model on the weighted training predictions
    xgb_meta_model.fit(weighted_train_preds.reshape(-1, 1), y_train)
    
    # Weighted test predictions using the same weights
    weighted_test_preds = (
        weights[0] * lgb_test_preds + weights[1] * xgb_test_preds
    )
    
    # Stack the test predictions for final output
    stacked_test_preds = weighted_test_preds.reshape(-1, 1)
    
    # Get final predictions from the neural network meta-model
    nn_final_predictions = nn_meta_model.predict(stacked_test_preds)
    
    # Get final predictions from the XGBoost meta-model
    xgb_final_predictions = xgb_meta_model.predict(stacked_test_preds)
    
    # Combine the predictions from both meta-models (could test weighted average of these too)
    final_predictions = (nn_final_predictions + xgb_final_predictions) / 2
    
    # Evaluate the model's performance
    mse = mean_squared_error(y_test, final_predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, final_predictions)
    r2 = r2_score(y_test, final_predictions)
    
    metrics = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }
    
    # Track the best weight combination and performance based on R² score
    if r2 > best_r2:
        best_r2 = r2
        best_weight_combination = weights
        best_metrics = metrics

# Print the best weight combination and corresponding performance metrics
print("\nBest Weights for Weighted Stacking Ensemble:")
print(f"LightGBM Weight: {best_weight_combination[0]}")
print(f"XGBoost Weight: {best_weight_combination[1]}")
print("\nBest Weighted Stacking Ensemble Performance:")
print(best_metrics)



Best Weights for Weighted Stacking Ensemble:
LightGBM Weight: 0.1
XGBoost Weight: 0.9

Best Weighted Stacking Ensemble Performance:
{'MSE': 7554026894.217134, 'RMSE': 86913.9050682751, 'MAE': 65157.73047185487, 'R2': 0.44041106485877135}


In [124]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor

# Define the range of weights to test for LightGBM and XGBoost
weight_range = np.linspace(0.05, 0.95, 19)  # Test weights from 0.05 to 0.95 for finer granularity

best_weight_combination = None
best_r2 = -np.inf
best_metrics = None

# Loop through weight combinations for LightGBM and XGBoost
for weight in weight_range:
    # Define weights for LightGBM and XGBoost
    weights = [weight, 1 - weight]
    
    # Create weighted train predictions
    weighted_train_preds = (
        weights[0] * lgb_train_preds + weights[1] * xgb_train_preds
    )
    
    # Define a wider grid for Ridge regularization
    param_grid_meta = {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 500]  # Wider range of regularization strength
    }
    
    # GridSearchCV for Ridge meta-model
    grid_meta = GridSearchCV(
        Ridge(),
        param_grid_meta,
        cv=10,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=1
    )
    
    grid_meta.fit(weighted_train_preds.reshape(-1, 1), y_train)
    best_meta_model = grid_meta.best_estimator_
    
    # Train meta-model on the weighted train predictions
    best_meta_model.fit(weighted_train_preds.reshape(-1, 1), y_train)
    
    # Create weighted test predictions
    weighted_test_preds = (
        weights[0] * lgb_test_preds + weights[1] * xgb_test_preds
    )
    
    stacked_test_preds = weighted_test_preds.reshape(-1, 1)
    final_predictions = best_meta_model.predict(stacked_test_preds)
    
    # Evaluate performance
    metrics = evaluate_model(y_test, final_predictions)
    r2 = metrics['R2']
    
    # Track best weight combination based on R2 score
    if r2 > best_r2:
        best_r2 = r2
        best_weight_combination = weights
        best_metrics = metrics

# Print the best weight combination and performance metrics
print("\nBest Weights for Weighted Stacking Ensemble:")
print(f"LightGBM Weight: {best_weight_combination[0]}")
print(f"XGBoost Weight: {best_weight_combination[1]}")
print("\nBest Weighted Stacking Ensemble Performance:")
print(best_metrics)


Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for each of 7 candidates, totalling 70 fits
Fitting 10 folds for eac

Now we will perform cross validation
Cross-validate the ensemble model on unseen data to ensure robustness.

In [113]:
# Load the dataset
california_houses = load_data("California_Houses.csv")

Successfully loaded data from California_Houses.csv.


In [114]:
# Check the column names in the new dataset
print(california_houses.columns)

# Identify missing or extra columns compared to the original dataset
required_columns = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
                    'total_bedrooms', 'population', 'households', 'median_income',
                    'ocean_proximity']
missing_columns = set(required_columns) - set(california_houses.columns)
extra_columns = set(california_houses.columns) - set(required_columns)

print("Missing columns:", missing_columns)
print("Extra columns:", extra_columns)


Index(['Median_House_Value', 'Median_Income', 'Median_Age', 'Tot_Rooms',
       'Tot_Bedrooms', 'Population', 'Households', 'Latitude', 'Longitude',
       'Distance_to_coast', 'Distance_to_LA', 'Distance_to_SanDiego',
       'Distance_to_SanJose', 'Distance_to_SanFrancisco'],
      dtype='object')
Missing columns: {'total_bedrooms', 'housing_median_age', 'longitude', 'median_income', 'population', 'latitude', 'total_rooms', 'ocean_proximity', 'households'}
Extra columns: {'Distance_to_coast', 'Median_Age', 'Median_House_Value', 'Median_Income', 'Households', 'Latitude', 'Distance_to_SanJose', 'Tot_Rooms', 'Distance_to_SanDiego', 'Distance_to_LA', 'Population', 'Distance_to_SanFrancisco', 'Tot_Bedrooms', 'Longitude'}


***Match Columns and Handle Missing Features***
We need to rename, transform, or drop certain columns.
The new dataset uses slightly different names for similar concepts.

In [115]:
column_mapping = {
    'Longitude': 'longitude',
    'Latitude': 'latitude',
    'Median_Age': 'housing_median_age',
    'Tot_Rooms': 'total_rooms',
    'Tot_Bedrooms': 'total_bedrooms',
    'Population': 'population',
    'Households': 'households',
    'Median_Income': 'median_income',
    'Median_House_Value': 'median_house_value'
}

# Apply the mapping
california_houses.rename(columns=column_mapping, inplace=True)


# we drop extra columns is consistency with the model's training data. 
# The ensemble model has only seen the original dataset's features during training. 
# Including new features at prediction time would confuse the model, as it has no learned parameters or relationships for those features.


extra_columns = ['Distance_to_coast', 'Distance_to_LA', 'Distance_to_SanDiego', 
                'Distance_to_SanJose', 'Distance_to_SanFrancisco']
california_houses.drop(columns=extra_columns, inplace=True)

# Assign a placeholder value
california_houses['ocean_proximity'] = 'NEAR BAY'




In [116]:
x_california_houses = california_houses.drop(['median_house_value'], axis=1)
y_california_houses = california_houses['median_house_value']

In [118]:
# Load preprocessing pipeline from saved model
with open('best_lgb_model_final.pkl', 'rb') as f:
    best_lgb_model = pickle.load(f)

preprocessing_pipeline = best_lgb_model.named_steps['preprocessing']

In [119]:
preprocessing_pipeline

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('outlier_handler',
                                                  OutlierHandler()),
                                                 ('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['longitude', 'latitude', 'housing_median_age',
                                  'total_rooms', 'total_bedrooms', 'population',
                                  'households', 'median_income',
                                  'bedroom_ratio', 'rooms_per_person',
                                  'income_per_household',
                                  'bedrooms_per_household'...
                                  'population_per_bedroom',
                                  'income_to_room_ratio',
                                  'income_per_age_of_housing',
                                  'household_room_interaction',

In [120]:
# Apply feature engineering as a standalone step to update `x_california_houses` with new features.
x_california_houses_fe = feature_engineering().fit_transform(x_california_houses)

# Redefine numerical and categorical features based on the transformed data.
numerical_features = x_california_houses_fe.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = x_california_houses_fe.select_dtypes(exclude=[np.number]).columns.tolist()

NameError: name 'feature_engineering' is not defined

In [ ]:
# Align columns in x_train and x_california_houses by adding missing columns with NaN values
missing_in_train = set(x_train.columns) - set(x_california_houses_fe.columns)
missing_in_test = set(x_california_houses_fe.columns) - set(x_train.columns)

# Add missing columns to x_california_houses and x_train with NaN values
for col in missing_in_train:
    x_california_houses_fe[col] = np.nan

for col in missing_in_test:
    x_train[col] = np.nan

# Reorder columns so that they match
x_california_houses = x_california_houses_fe[x_train.columns]


In [40]:
# Apply preprocessing to new data
processed_new_data = preprocessing_pipeline.transform(x_california_houses)

NameError: name 'x_california_houses' is not defined

In [ ]:
# Load the saved models and weights
with open('best_xgb_model_final.pkl', 'rb') as f:
    best_xgb_model = pickle.load(f)

with open('optimal_weights.pkl', 'rb') as f:
    optimal_weights = pickle.load(f)

# Generate predictions for the new data
lgb_predictions = best_lgb_model.named_steps['model'].predict(processed_new_data)
xgb_predictions = best_xgb_model.named_steps['model'].predict(processed_new_data)

# Weighted predictions
final_predictions = (
    optimal_weights[0] * lgb_predictions + optimal_weights[1] * xgb_predictions
)

# Print predictions
print("Ensemble model predictions for new data:")
print(final_predictions)


In [39]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.base import BaseEstimator, RegressorMixin
from lightgbm import LGBMRegressor

# Set LightGBM parameters to resolve warnings (if re-training or initializing)
lightgbm_params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'min_data_in_leaf': 10  # Ensure consistency
}

# Split into features and target
X_new = processed_new_data
y_new = y_california_houses

# Define a scoring function
scorer = make_scorer(mean_squared_error, squared=False)

# Define ensemble prediction function
def ensemble_predict(X):
    lgb_preds = best_lgb_model.named_steps['model'].predict(X)
    xgb_preds = best_xgb_model.named_steps['model'].predict(X)
    return optimal_weights[0] * lgb_preds + optimal_weights[1] * xgb_preds

# Wrap ensemble logic in a scikit-learn-compatible estimator
class EnsembleRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, predict_function):
        self.predict_function = predict_function

    def fit(self, X, y):
        return self  # Pre-trained models; no fitting required

    def predict(self, X):
        return self.predict_function(X)

# Create a scikit-learn compatible ensemble regressor
ensemble_model = EnsembleRegressor(predict_function=ensemble_predict)

# Perform cross-validation
cv_scores = cross_val_score(ensemble_model, processed_new_data, y_california_houses, scoring=scorer, cv=5)

# Output results
print(f"Cross-validation RMSE: {cv_scores.mean()} ± {cv_scores.std()}")


NameError: name 'processed_new_data' is not defined

 Finalize the Weighted Averaging Ensemble
We'll save the optimal weights and ensure they are used for final predictions.